In [21]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80
)

def add_message(input, output):
    memory.save_context({"input" : input},{"output" : output})

with get_openai_callback() as usage:
    add_message("Hi I'm Ethan, I live in South Korea", "Wow that is so cool!")
    add_message("I'm a software engineer.", "Very interesting to hear that!")
    result = memory.load_memory_variables({"input" : "Who is Ethan?"})
    print("Usage during message addition:", usage)
    print(result)




Usage during message addition: Tokens Used: 1505
	Prompt Tokens: 1482
	Completion Tokens: 23
Successful Requests: 3
Total Cost (USD): $0.002269
{'history': 'On Ethan: Ethan lives in South Korea. Ethan is a software engineer.'}
